# Phylo-CTF-analysis-ASV

**Note**: This notebook assumes you have installed [QIIME2](https://qiime2.org/) using one of the procedures in the [install documents](https://docs.qiime2.org/2020.2/install/). This tutorial also assumed you have installed, [Qurro](https://github.com/biocore/qurro), [DEICODE](https://github.com/biocore/DEICODE), and [gemelli](https://github.com/biocore/gemelli).

First, we will make a tutorial directory and download the data above and move the files to the `Single-tooth-ECC/Data`， the results will be saved in the `Single-tooth-ECC/Results/Dist_matrix`:

```bash
mkdir -p ../../Results/Dist_matrix
```

First we will import our data with the QIIME2 Python API. 


In [1]:
# ! pip install gemelli
# ! pip install deicod
# ! pip install qurro
# ! pip install empress

In [2]:
!mkdir -p ../../Results/Feature_table
!mkdir -p ../../Results/Dist_matrix

! qiime tools import \
    --type 'FeatureTable[Frequency]' \
    --input-path ../../Data/1867/taxonomy/1867_taxonomic_ASV_count_table.biom \
    --output-path ../../Results/Feature_table/1867_taxonomic_ASV_count_table.qza

! qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-path ../../Data/1867/taxonomy/1867_taxonomic_feature_taxon_SILVA.txt \
  --output-path ../../Results/Feature_table/1867_taxonomic_feature_taxonomy.qza

Imported ../../Data/1867/taxonomy/1867_taxonomic_ASV_count_table.biom as BIOMV210DirFmt to ../../Results/Feature_table/1867_taxonomic_ASV_count_table.qza
Imported ../../Data/1867/taxonomy/1867_taxonomic_feature_taxon_SILVA.txt as TSVTaxonomyDirectoryFormat to ../../Results/Feature_table/1867_taxonomic_feature_taxonomy.qza


In [3]:
import os
import warnings
import qiime2 as q2
from qiime2.plugins.deicode.actions import rpca
from qiime2.plugins.emperor.actions import (plot, biplot)
from qiime2.plugins.diversity.actions import (beta_phylogenetic, pcoa, beta_group_significance)

# hide pandas Future/Deprecation Warning(s) for tutorial
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.simplefilter(action='ignore', category=FutureWarning)

# import table(s)
table = q2.Artifact.load('../../Results/Feature_table/1867_taxonomic_ASV_count_table.qza')
# import metadata
metadata = q2.Metadata.load('../../Data/1867/1867_metadata.txt')
# import tree
tree = q2.Artifact.load('../../Data/1867/taxonomy/1867_taxonomic_phylogenetic_tree.qza')
# import taxonomy
taxonomy = q2.Artifact.load('../../Results/Feature_table/1867_taxonomic_feature_taxonomy.qza')
# make directory to store results
output_path = '../../Results/Dist_matrix/1867_taxonomic_phylo_rPCA'
if not os.path.exists(output_path): 
    os.mkdir(output_path)


Next, we will demonstrate the issues with using conventional dimensionality reduction methods on time series data. To do this we will perform PCoA dimensionality reduction on weighted and unweighted UniFrac $\beta$-diversity distances. We will also run Aitchison Robust PCA with _DEICODE_ which is built on the same framework as CTF but does not account for repeated measures.


### Sample filtering based on the metadata

In [4]:
import pandas as pd
from qiime2.plugins.feature_table.actions import filter_seqs
from qiime2.plugins.feature_table.actions import filter_samples
from qiime2.plugins.feature_table.actions import summarize

In [5]:
table.view(pd.DataFrame).shape

(1867, 18039)

In [6]:
taxonomy.view(q2.Metadata)

Metadata
--------
18054 IDs x 5 columns
Taxon:           ColumnProperties(type='categorical', missing_scheme='blank')
Confidence:      ColumnProperties(type='categorical', missing_scheme='blank')
ASV_ID:          ColumnProperties(type='categorical', missing_scheme='blank')
Brief_Taxon:     ColumnProperties(type='categorical', missing_scheme='blank')
Brief_Taxon_ASV: ColumnProperties(type='categorical', missing_scheme='blank')

Call to_dataframe() for a tabular representation.

# Phylogenetic rPCA

In [7]:
from qiime2.plugins.gemelli.actions import phylogenetic_rpca_with_taxonomy
#table = filter_features(table, tree).filtered_table
phylo_rpca_results = phylogenetic_rpca_with_taxonomy(table, tree, taxonomy.view(q2.Metadata),
                                             min_feature_frequency=10)
phylo_rpca_results

Results (name = value)
---------------------------------------------------------------------------------------------------------------
biplot              = <artifact: PCoAResults % Properties('biplot') uuid: 0bcd30be-7aa1-4832-87fe-591e863a294c>
distance_matrix     = <artifact: DistanceMatrix uuid: a622edb6-f929-4fe3-bd75-f7a60b3f0336>
counts_by_node_tree = <artifact: Phylogeny[Rooted] uuid: 942b7df8-726e-4fcf-87b0-69dc00398835>
counts_by_node      = <artifact: FeatureTable[Frequency] uuid: df8c8eec-1535-4166-9dc9-3b95f9eae10d>
t2t_taxonomy        = <artifact: FeatureData[Taxonomy] uuid: 3e424300-f63c-4ec4-a75d-9f55ca9b17ee>

In [8]:
phylo_rpca_results.biplot.save(os.path.join(output_path, 'phylo_rpca.biplot.qza'))
phylo_rpca_results.distance_matrix.save(os.path.join(output_path, 'phylo_rpca.distance_matrix.qza'))
phylo_rpca_results.counts_by_node_tree.save(os.path.join(output_path, 'phylo_rpca.counts_by_node_tree.qza'))
phylo_rpca_results.counts_by_node.save(os.path.join(output_path, 'phylo_rpca.counts_by_node.qza'))
phylo_rpca_results.t2t_taxonomy.save(os.path.join(output_path, 'phylo_rpca.t2t_taxonomy.qza'))

'../../Results/Dist_matrix/1867_taxonomic_phylo_rPCA/phylo_rpca.t2t_taxonomy.qza'

In [9]:
phylo_rpca_biplot_emperor = biplot(phylo_rpca_results.biplot, metadata)
phylo_rpca_biplot_emperor.visualization.save(os.path.join(output_path, 'phylo_RPCA-biplot.qzv'))


'../../Results/Dist_matrix/1867_taxonomic_phylo_rPCA/phylo_RPCA-biplot.qzv'

In [10]:
from skbio.stats.distance import DistanceMatrix
distance_matrix = phylo_rpca_results.distance_matrix.view(DistanceMatrix)
distance_df = distance_matrix.to_data_frame()
distance_df.to_csv(os.path.join(output_path, '1867_taxonomic_phylo_rpca_dist.txt'), sep='\t', index=True)

In [11]:
from skbio import OrdinationResults

sample_coords = phylo_rpca_results.biplot.view(OrdinationResults).samples
sample_coords = sample_coords.rename(columns={0: 'PC1', 1: 'PC2', 2: 'PC3'})
sample_coords.index.name = 'SampleID'
sample_coords.head(5)
sample_coords.to_csv(os.path.join(output_path,'1867_taxonomic_phylo_rpca_sample_coordinates.tsv'), sep = "\t", index=True)


In [12]:
# combine feature metadata
from skbio import OrdinationResults

phylo_rpca_taxonomy = phylo_rpca_results.t2t_taxonomy.view(q2.Metadata).to_dataframe()
phylo_rpca_feature_loadings = phylo_rpca_results.biplot.view(OrdinationResults).features.rename({0:'PC1', 1:"PC2", 2:"PC3"}, axis=1)
phylo_rpca_taxonomy_and_loadings = pd.concat([phylo_rpca_taxonomy, phylo_rpca_feature_loadings], axis=1)
phylo_rpca_taxonomy_and_loadings.index.name = 'featureid'
phylo_rpca_taxonomy_and_loadings.head(5)

,Taxon,PC1,PC2,PC3
featureid,,,,
n750,d__Bacteria; p__; c__; o__; f__; g__; s__,-0.035152,-0.002255,0.103146
n748,d__Bacteria; p__; c__; o__; f__; g__; s__,-0.027883,0.000388,0.075912
n382,d__Bacteria; p__Actinobacteriota; c__; o__; f_...,-0.051136,0.014808,0.076259
n77,d__Bacteria; p__Actinobacteriota; c__Actinobac...,-0.046485,0.013843,0.057868
GTGGGGAATATTGCACAATGGGCGCAAGCCTGATGCAGCGACGCCGCGTGCGGGATGGAGGCCTTCGGGTTGTAAACCGCTTTTATAGGGGGGCAAGCTATGCCTGTGTGGTGTGGTGAGTGGACTTTATGAATAAGCACCGGCTAACTACGTGCCAGCAGCCGCGGTAATACGTAGGGTGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGGGCTCGTAGGCGGTTTGTTGCGTCTGGTGTGAAAGCTT,d__Bacteria; p__Actinobacteriota; c__Actinobac...,0.001634,0.043387,0.033252
